In [1]:
import json
import mysql.connector
import psycopg2


/Users/Mat/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
mydb = psycopg2.connect(
  host="127.0.0.1",
  port="32769",
  user="postgres",
  password="mysecretpassword",
  dbname="innodb"
)


mycursor = mydb.cursor()

In [3]:
# mycursor.execute("SELECT id FROM commercial_premise WHERE name = %s", ("AHOAN",))
# records = mycursor.fetchone()

# Create tables

In [4]:
# Main table
sql_main_table = """
    CREATE TABLE IF NOT EXISTS commercial_premise (
        id SERIAL,
        name VARCHAR(100) NOT NULL UNIQUE,
        zip_code INT NOT NULL,
        coordinates VARCHAR(25),
        telephone_number VARCHAR(25),
        opening_hours VARCHAR(100),
        type VARCHAR(20) NOT NULL,
        score FLOAT(2) DEFAULT 0.0,
        total_scores INT DEFAULT 0,
        price_range VARCHAR(5),
        style VARCHAR(20),
        address VARCHAR(40) NOT NULL,
        date DATE NOT NULL,
        PRIMARY KEY(ID)
    )
"""

sql_comments = """
    CREATE TABLE IF NOT EXISTS commercial_premise_comments (
        id SERIAL,
        commercial_premise_id INT NOT NULL,
        content VARCHAR(600),
        date DATE NOT NULL,
        PRIMARY KEY(id),
        FOREIGN KEY (commercial_premise_id)
            REFERENCES commercial_premise(id)
            ON DELETE CASCADE
            ON UPDATE CASCADE
    )
"""

sql_ocupation = """
    CREATE TABLE IF NOT EXISTS commercial_premise_occupation (
        id SERIAL,
        commercial_premise_id INT NOT NULL,
        week_day VARCHAR(50) NOT NULL,
        time_period CHAR(50) NOT NULL,
        occupation FLOAT DEFAULT 0.0,
        date DATE NOT NULL,
        PRIMARY KEY(id),
        FOREIGN KEY (commercial_premise_id)
            REFERENCES commercial_premise(id)
            ON DELETE CASCADE
            ON UPDATE CASCADE
    )
"""

# mycursor.execute("DROP TABLE commercial_premise_occupation")
# mycursor.execute("DROP TABLE commercial_premise_comments")
# mycursor.execute("DROP TABLE commercial_premise")

mycursor.execute(sql_main_table)
mycursor.execute(sql_comments)
mycursor.execute(sql_ocupation)
mydb.commit()
mycursor.execute("SELECT * FROM pg_stat_user_tables;")
mycursor.fetchall()

[(17213,
  'public',
  'commercial_premise',
  3,
  0,
  14,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  None,
  None,
  None,
  None,
  0,
  0,
  0,
  0),
 (17241,
  'public',
  'commercial_premise_occupation',
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  None,
  None,
  None,
  None,
  0,
  0,
  0,
  0),
 (17225,
  'public',
  'commercial_premise_comments',
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  None,
  None,
  None,
  None,
  0,
  0,
  0,
  0)]

In [5]:
# from pprint import pprint
# sql1 = "DESCRIBE commercial_premise"
# sql2 = "DESCRIBE commercial_premise_comments"
# sql3 = "DESCRIBE commercial_premise_occupation"
# mycursor.execute(sql1)
# pprint(mycursor.fetchall())
# mycursor.execute(sql2)
# pprint(mycursor.fetchall())
# mycursor.execute(sql3)
# pprint(mycursor.fetchall())

# Read data

In [6]:
with open("../data.json_OLD", "r") as file:
    sample = json.loads(file.read())

In [7]:
sample.keys()

dict_keys(['AHOAN', 'Bar Restaurante Víctor', 'Huevos Y Tortilla', 'con B de bilbao', 'Restaurante Lurrina', 'Restaurante-Cocktail Yamike', 'Café Bar Bilbao'])

In [8]:
len(sample.keys())

7

# Format data

In [9]:
def decompose_occupancy_data(occupancy_levels):
    occupancy = {
        "lunes": {},
        "martes": {},
        "miercoles": {},
        "jueves": {},
        "viernes": {},
        "sabado": {},
        "domingo": {}
    }
    for week_day, occupancy_levels in occupancy_levels.items():
        if(occupancy_levels is not None):
            for occupancy_level in occupancy_levels:
                if(occupancy_level is not None):
                    try:
                        base = occupancy_level.split(":")[1:]
                        occupancy[week_day].update({
                            base[1].split(")")[0].strip(): float(base[0].split("\xa0%")[0])
                        })
                    except:
                        pass
    return occupancy
        

In [10]:
def write_to_db(element, zip_code, premise_type, mydb, mycursor, date):

    mycursor.execute("SELECT id FROM commercial_premise WHERE name = %s", (element["name"],))
    records = mycursor.fetchone()
    if(records is None):
        #Store element
        if element["address"] is not None:
            compressed_address = [item.strip() for item in element["address"].split("·")]
            if(len(compressed_address) == 3):
                price_range = compressed_address[0]
                style = compressed_address[1]
                address = compressed_address[2]
            else:
                price_range = None
                style = compressed_address[0]
                address = compressed_address[1]
        else:
            price_range = None
            style = None
            address = None
        sql = """
            INSERT INTO commercial_premise 
                (name, zip_code, coordinates, telephone_number, openig_hours, type, score, total_scores, price_range, style, address, date) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
        keys = element.keys()
        values = (
            element["name"],
            zip_code,
            element["coordinates"] if "coordinates" in keys else None,
            element["telephone_number"] if "telephone_number" in keys else None,
            ",".join(element["opennig_hours"]) if "opennig_hours" in keys and element["opennig_hours"] is not None else None,
            premise_type,
            element["score"],
            element["total_scores"],
            price_range,
            style,
            address,
            date
        )
        mycursor.execute(sql, values)
        mydb.commit()
        element_id = mycursor.lastrowid
    else:
        element_id = records[0]
    #Store comments
    if("comments" in keys):
        sql = """
                INSERT INTO commercial_premise_comments
                (commercial_premise_id, content, date)
                VALUES (%s, %s, %s)
            """
        values = []
        for comment in element["comments"]:
            values.append((element_id, comment, date))
        mycursor.executemany(sql, values)
    #Store occupancy data
    sql = """
        INSERT INTO commercial_premise_occupation
        (
            commercial_premise_id, week_day, time_period, occupation, date
        )
        VALUES (%s, %s, %s, %s, %s)
    """
    if("occupancy" in keys):
        values = []
        for week_day, content in decompose_occupancy_data(element["occupancy"]).items():
            if content is not None and content != {}:
                for key,value in content.items():
                    values.append((element_id, week_day, key, value, date))
        mycursor.executemany(sql, values)
    mydb.commit()

In [11]:
import time

In [12]:
from datetime import datetime

In [13]:
t1 = time.time()
for name, element in sample.items():
    print(name)
    try:
        write_to_db(element, "48005", "restaurante", mydb, mycursor, datetime.now().date())
    except Exception as e:
        mydb.rollback()
        print("ERROR ", name, e)
t2 = time.time()
print(t2-t1)

AHOAN
ERROR  AHOAN column "openig_hours" of relation "commercial_premise" does not exist
LINE 3: ...  (name, zip_code, coordinates, telephone_number, openig_hou...
                                                             ^

Bar Restaurante Víctor
ERROR  Bar Restaurante Víctor column "openig_hours" of relation "commercial_premise" does not exist
LINE 3: ...  (name, zip_code, coordinates, telephone_number, openig_hou...
                                                             ^

Huevos Y Tortilla
ERROR  Huevos Y Tortilla column "openig_hours" of relation "commercial_premise" does not exist
LINE 3: ...  (name, zip_code, coordinates, telephone_number, openig_hou...
                                                             ^

con B de bilbao
ERROR  con B de bilbao column "openig_hours" of relation "commercial_premise" does not exist
LINE 3: ...  (name, zip_code, coordinates, telephone_number, openig_hou...
                                                             ^

Restaurant

In [ ]:
'AHOAN', 'Bar Restaurante Víctor', 'Huevos Y Tortilla', 'con B de bilbao', 'Restaurante Lurrina', 'Restaurante-Cocktail Yamike', 'Café Bar Bilbao'

In [18]:
datetime.now().date()

datetime.date(2020, 4, 8)

In [15]:
mycursor.execute("SELECT * FROM commercial_premise;")
mycursor.fetchall()

InternalError: current transaction is aborted, commands ignored until end of transaction block
